In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from PIL import Image

from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

import pandas as pd

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        print (image.shape)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    print(image_files)
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def make_mask_img(ori_img, mask_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,2] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [ ]:
def make_2_mask_img(ori_img, mask_img_1, mask_img_2):
    
    mask_img_rgb = np.zeros((mask_img_1.shape[0], mask_img_1.shape[1], 3), np.float32)
    mask_img_rgb[:,:,0] = mask_img_2[:,:,0]
    mask_img_rgb[:,:,1] = mask_img_1[:,:,0] + mask_img_2[:,:,0]
    mask_img_rgb[:,:,2] = mask_img_1[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img + 0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [ ]:
ori_imgs, _ = load_X_gray("Y:/DeepLearningData/research_010_NIH3T3/superhuman_lamellar/original/")

In [ ]:
suga_imgs_1, _ = load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\superhuman_lamellar\suga\lamellar_1")
suga_imgs_3, _ = load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\superhuman_lamellar\suga\lamellar_3")


In [ ]:
suga_imgs_1 *= 255 * 255
suga_imgs_3 *= 255 *255

In [ ]:
nakamura_imgs_1, _ = load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\superhuman_lamellar\nakamura\lamellar_1")
nakamura_imgs_3, _ = load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\superhuman_lamellar\nakamura\lamellar_3")


In [ ]:
nakamura_imgs_1 *= 255 * 255
nakamura_imgs_3 *= 255 *255

In [ ]:
DL_imgs_1, _ =  load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\superhuman_lamellar\DL_only\lamellar_1")
DL_imgs_3, _ =  load_Y_gray_with_gaussian_and_numeric(r"Y:\DeepLearningData\research_010_NIH3T3\superhuman_lamellar\DL_only\lamellar_3")


In [ ]:
DL_imgs_1 *= 255 * 255
DL_imgs_3 *= 255 *255

In [ ]:
N = 47
ori_imgs_=make_2_mask_img(ori_imgs[N], np.zeros((109, 115,  1)),np.zeros((109, 115,  1)))
masked_suga_imgs = make_2_mask_img(ori_imgs[N], suga_imgs_1[N], suga_imgs_3[N])
masked_nakamura_imgs = make_2_mask_img(ori_imgs[N], nakamura_imgs_1[N], nakamura_imgs_3[N])
masked_DL_imgs = make_2_mask_img(ori_imgs[N], DL_imgs_1[N], DL_imgs_3[N])

In [ ]:
def expand_img(img):
    """
        OxO ->360x360
    """
    large_img = np.zeros(
        (450, 450, 3), np.float32
    )
    
    N = 450 // img.shape[0]
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for rgb in range(img.shape[2]):
                large_img[i*N :  (i+1)*N, j*N : (j+1)*N, rgb] = img[i, j,  rgb]
                
    return large_img

In [ ]:
out_dir= "Z:/paper_data/2021_cristae/sup_fig_001_superhuman_expand_imgs"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/xy_em_001_img.png",
    expand_img(denormalize_y(ori_imgs_[25:70,10:55]))
)
cv2.imwrite(
    f"{out_dir}/xy_dl_001_img.png",
    expand_img(denormalize_y(masked_DL_imgs[25:70,10:55]))
)
cv2.imwrite(
    f"{out_dir}/xy_nk_001_img.png",
    expand_img(denormalize_y(masked_nakamura_imgs[25:70,10:55]))
)
cv2.imwrite(
    f"{out_dir}/xy_ss_001_img.png",
    expand_img(denormalize_y(masked_suga_imgs[25:70,10:55]))
)

cv2.imwrite(
    f"{out_dir}/xy_em_001_mag_img.png",
    expand_img(denormalize_y(ori_imgs_[45:60,22:37]))
)
cv2.imwrite(
    f"{out_dir}/xy_dl_001_mag_img.png",
    expand_img(denormalize_y(masked_DL_imgs[45:60,22:37]))
)
cv2.imwrite(
    f"{out_dir}/xy_nk_001_mag_img.png",
    expand_img(denormalize_y(masked_nakamura_imgs[45:60,22:37]))
)
cv2.imwrite(
    f"{out_dir}/xy_ss_001_mag_img.png",
    expand_img(denormalize_y(masked_suga_imgs[45:60,22:37]))
)

In [ ]:
N = 27
ORI = ori_imgs.transpose(2, 0, 1, 3)[N]
ORI_ = make_2_mask_img(
    ori_imgs.transpose(2, 0, 1, 3)[N], 
    np.zeros(suga_imgs_1.transpose(2, 0, 1, 3)[N].shape), 
    np.zeros(suga_imgs_3.transpose(2, 0, 1, 3)[N].shape)
)
masked_suga_imgs = make_2_mask_img(ori_imgs.transpose(2, 0, 1, 3)[N], suga_imgs_1.transpose(2, 0, 1, 3)[N], suga_imgs_3.transpose(2, 0, 1, 3)[N])
masked_nakamura_imgs = make_2_mask_img(ori_imgs.transpose(2, 0, 1, 3)[N], nakamura_imgs_1.transpose(2, 0, 1, 3)[N], nakamura_imgs_3.transpose(2, 0, 1, 3)[N])
masked_DL_imgs = make_2_mask_img(ori_imgs.transpose(2, 0, 1, 3)[N], DL_imgs_1.transpose(2, 0, 1, 3)[N], DL_imgs_3.transpose(2, 0, 1, 3)[N])

In [ ]:
out_dir= "Z:/paper_data/2021_cristae/sup_fig_001_superhuman_expand_imgs"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/yz_em_001_img.png",
    expand_img(denormalize_y(ORI_[30:80,30:80]))
)
cv2.imwrite(
    f"{out_dir}/yz_dl_001_img.png",
    expand_img(denormalize_y(masked_DL_imgs[30:80,30:80]))
)
cv2.imwrite(
    f"{out_dir}/yz_nk_001_img.png",
    expand_img(denormalize_y(masked_nakamura_imgs[30:80,30:80]))
)
cv2.imwrite(
    f"{out_dir}/yz_ss_001_img.png",
    expand_img(denormalize_y(masked_suga_imgs[30:80,30:80]))
)

cv2.imwrite(
    f"{out_dir}/yz_em_001_mag_img.png",
    expand_img(denormalize_y(ORI_[45:60,50:65]))
)
cv2.imwrite(
    f"{out_dir}/yz_dl_001_mag_img.png",
    expand_img(denormalize_y(masked_DL_imgs[45:60,50:65]))
)
cv2.imwrite(
    f"{out_dir}/yz_nk_001_mag_img.png",
    expand_img(denormalize_y(masked_nakamura_imgs[45:60,50:65]))
)
cv2.imwrite(
    f"{out_dir}/yz_ss_001_mag_img.png",
    expand_img(denormalize_y(masked_suga_imgs[45:60,50:65]))
)

In [ ]:
ORI = denormalize_x(ORI_[47,51:60, 0]/0.6-0.75)

dl = np.array([np.nan] * 9)
dl[2:5] = denormalize_x(ORI_[47,51:60, 0]/0.6-0.75)[2:5]

nk= np.array([np.nan] * 9)
nk[1:5] = denormalize_x(ORI_[47,51:60, 0]/0.6-0.75)[1:5]

ss= np.array([np.nan] * 9)
ss[2:7] = denormalize_x(ORI_[47,51:60, 0]/0.6-0.75)[2:7]

In [ ]:
ORI_rev= 125 - ORI
dl_rev=125-dl
nk_rev=125-nk
ss_rev=125-ss
X=np.array([i for i in range(9)])

In [ ]:
plt.figure(figsize=(10, 8))

#plt.title("DL")

plt.bar(X,ORI_rev,color="black", width=1)
plt.bar(X, dl_rev, color="aqua", width=1)
#plt.axis([0, 8, 130,　65 ])
plt.gca().axes.xaxis.set_visible(False)
plt.gca().axes.yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.savefig(f"{out_dir}/xy_dl_bar.png")

In [ ]:
plt.figure(figsize=(10, 8))

#plt.title("DL")
plt.bar(X,ORI_rev,color="black", width=1)
plt.bar(X, nk_rev, color="aqua", width=1)
plt.gca().axes.xaxis.set_visible(False)
plt.gca().axes.yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.savefig(f"{out_dir}/xy_nk_bar.png")

In [ ]:
plt.figure(figsize=(10, 8))

#plt.title("DL")
plt.bar(X,ORI_rev,color="black", width=1)
plt.bar(X, ss_rev, color="aqua", width=1)
plt.gca().axes.xaxis.set_visible(False)
plt.gca().axes.yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.savefig(f"{out_dir}/xy_ss_bar.png")